# Загрузка библиотек

In [ ]:
import os
import json

import numpy as np

from glob import glob
from transformers import MarkupLMFeatureExtractor, MarkupLMProcessor, MarkupLMForTokenClassification
from bs4 import BeautifulSoup
from torch.utils.data import Dataset, random_split, DataLoader
from tqdm import tqdm

/home/ubuntu/.cache/pypoetry/virtualenvs/markupclassification-_GlTHi5g-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Configuration

In [ ]:
batch_size = 50

allowed_labels = ["title", "short_text", "date", "time", "tag", "short_title", "author"]

label2id = {label: idx+1 for idx, label in enumerate(allowed_labels)}
label2id["OTHER"] = 0

id2label = {idx+1: label for idx, label in enumerate(allowed_labels)}
id2label[0] = "OTHER"

# Загрузка данных

In [ ]:
def load_from_folder(folder_path : str):
    '''
        This function loading all json files from folder.
        Each file contains dict with labels and its values.
        Each file must contains "html" label with its html code. 
        Each file must contains "xpaths" label with its labeled xpaths list. 
        
    '''
    extractor = MarkupLMFeatureExtractor()
    
    folder_path = os.path.abspath(folder_path)
    files_path = glob(os.path.join(folder_path, "*.json"))
    
    data = []
    
    for file_path in tqdm(files_path):
        print(file_path)
        with open(file_path) as file:
            info = json.load(file)
            
        html = info["html"]
        labeled_xpaths = info["labeled_xpaths"]

        encoding = extractor(html)
            
        
        labels = []
        for xpath in encoding["xpaths"][0]:
            if xpath in labeled_xpaths:
                labels.append(label2id[labeled_xpaths[xpath]])
            else:
                labels.append(0)


        # print(len(labels))
        # print([_ for _ in labels if _ != 0])
        
        if (len([_ for _ in labels if _ != 0]) == 0):
            print(file_path)
        
        labels = [labels]
        # print(len(encoding['nodes'][0]), len(encoding['xpaths'][0]), len(labels[0]))
        data.append({'nodes': encoding['nodes'],
                     'xpaths': encoding['xpaths'],
                     'node_labels': labels,
                     'html': html})
        
    return data
    

In [ ]:
train_data = load_from_folder("test_dataset/train_part")
valid_data = load_from_folder("test_dataset/test_part")

  0%|          | 0/1980 [00:00<?, ?it/s]

/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/24tnews.ru_80.json


  0%|          | 1/1980 [00:14<7:56:38, 14.45s/it]

/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/piter98.ru_24.json


  0%|          | 2/1980 [00:14<3:19:15,  6.04s/it]

...

/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/gorodkovrov.ru_69.json


 12%|█▏        | 244/1980 [01:26<09:18,  3.11it/s]

/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/kaluganews.com_38.json
/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/www.newsler.ru_54.json


 12%|█▏        | 245/1980 [01:26<09:20,  3.10it/s]

/home/ubuntu/Documents/MarkupClassification/test_dataset/train_part/gazeta.a42.ru_6.json


In [ ]:
print("Train size : ", len(train_data))
print("Test size : ", len(valid_data))
print("Train proportion : ", (len(valid_data) + len(train_data)) / len(train_data))

In [ ]:
idx = 0
for node, label in zip(valid_data[idx]['nodes'][0], valid_data[idx]['node_labels'][0]):
  if id2label[label] != 'OTHER':
    print(node, id2label[label])
     

# Инициалиация датасета

In [ ]:
class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None):
        self.data = data
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx]
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length", truncation=True, return_tensors="pt")

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

In [ ]:
processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base", truncation = True)
processor.parse_html = False

train_set = MarkupLMDataset(data=train_data, processor=processor)
valid_set = MarkupLMDataset(data=valid_data, processor=processor)

In [ ]:
example = valid_set[0]
for k,v in example.items():
  print(k,v.shape)

In [ ]:
processor.decode(example['input_ids'])

In [ ]:
for id, label in zip(example['input_ids'].tolist(), example['labels'].tolist()):
    # if label != -100:
    #     print(processor.decode([id]), label)
    if label == 1:
        print(processor.decode([id]), label)

In [ ]:
train_dataloader = DataLoader(train_set, batch_size=3, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=3, shuffle=True)

In [ ]:
model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=id2label, label2id=label2id)

In [ ]:
label_list = ["B-" + x for x in list(id2label.values())]

In [ ]:
import evaluate

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(metric, return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

# TRAIN

In [ ]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm
from sklearn.metrics import classification_report, f1_score

optimizer = AdamW(model.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

model.train()
print(device)
for epoch in range(10):
    for batch in tqdm(train_dataloader):
        # get the inputs;
        inputs = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        preds, refs = get_labels(predictions, labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )

    train_metric = compute_metrics(metric)
    print(f"Epoch {epoch}:", train_metric)
      


# TEST

In [ ]:
model.eval()
print(device)

test_metric = evaluate.load("seqeval")

for batch in tqdm(valid_dataloader):
    # get the inputs;
    inputs = {k:v.to(device) for k,v in batch.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = outputs.logits.argmax(dim=-1)
    labels = batch["labels"]
    preds, refs = get_labels(predictions, labels)
    test_metric.add_batch(
        predictions=preds,
        references=refs,
    )

eval_metric = compute_metrics(test_metric)
print("TESTING RESULT :", eval_metric)